In [1]:
!pip install rmsd
!pip install wget
import numpy as np
import rmsd
import wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=1ac4a47d565a37a90e553445d295ffb37400ac249e2e5417efa79e7a0bc7cc40
  Stored in directory: c:\users\sina-pc\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [2]:
pdbIDs=['2JF3.pdb', '2QIV.pdb', '2LAO.pdb', '1PDA.pdb', '1A11.pdb']

for i in range(len(pdbIDs)):
    url = "https://files.rcsb.org/download/" + pdbIDs[i]
    wget.download(url)

-1 / unknown

In [6]:
def read_pdb(file_path):
    rows = [] # First element is dummy variable will be removed.
    with open(file_path, "r") as file:
        linen= 1
        for line in file:
            row = line.strip().split()
            if linen > 1:
                if row[0] == "ATOM" and row[2] == "CA" and row[5] != rows[-1][5] and row[5].isdigit():
                    rows.append(row)
            else:
                if row[0] == "ATOM" and row[2] == "CA" and row[5].isdigit():
                    rows.append(row)
                    linen+=1

        for i,row in enumerate(rows):
            rows[i] = np.array(row[6:9]).astype(float)
    return rows


In [7]:
def calculate_rmsd(A, B):
  return np.sqrt(((np.array(A) - np.array(B))**2).sum() / len(A)) # Calculation of RMSD from Mehmet Gonen's codes DASC521

In [8]:
mata = read_pdb(pdbIDs[0])
matb = read_pdb(pdbIDs[1])

mata = mata- np.mean(mata, axis = 0) # move centers to the origin
matb = matb- np.mean(matb, axis = 0) # move centers to the origin

U = rmsd.kabsch(mata, matb) # super imposition function
mata = np.dot(mata, U) # super impose first prot on the second

print(f"RMSD: {calculate_rmsd(mata, matb)}")


RMSD: 0.2758513169989861


In [9]:

mata = read_pdb(pdbIDs[2])[92:187]
matb = read_pdb(pdbIDs[3])[102:197]

mata -= np.mean(mata, axis = 0) # move centers to the origin
matb -= np.mean(matb, axis = 0) # move centers to the origin

U = rmsd.kabsch(mata, matb) # suoer imposition function
mata = np.dot(mata, U) # super impose first prot on the second

print(f"RMSD: {calculate_rmsd(mata, matb)}")


RMSD: 15.526340371058206


In [10]:
mata = read_pdb(pdbIDs[2])
matb = read_pdb(pdbIDs[3])
RMSDs = [] # saving RMSD for each iteration
for i in range(abs(len(mata)-len(matb))):
    matbc = matb[i:(len(mata)+i)].copy()

    mata -= np.mean(mata, axis = 0)
    matbc -= np.mean(matbc, axis = 0)

    U = rmsd.kabsch(mata, matbc)
    mata = np.dot(mata, U)
    RMSDs.append(calculate_rmsd(mata, matbc))
    
  
print(RMSDs)   

print("\nGlobal Alignment RMSD is:", min(RMSDs))

[13.094372818454994, 13.699217132398747, 14.362976678946728, 15.076765622505418, 15.830481867131258, 16.556693577014048, 17.198623500298897, 17.803871631670095, 18.325512767755132, 18.746651770639474, 19.075891113718132, 19.313835231985454, 19.470913425458466, 19.59509976123791, 19.687540008113054, 19.71752706106357, 19.748231013293516, 19.82922578442605, 19.932355094196335, 20.011908969784795, 20.173290354975776, 20.408214606263986, 20.60517467000081, 20.794670383496285, 21.020004238663592, 21.25090335238693, 21.412869560098613, 21.539037727128704, 21.653125674929473, 21.68967912869312, 21.638986709014244, 21.558792071432155, 21.516491186883485, 21.453166684215493, 21.388597863347933, 21.340285529925378, 21.3456237253631, 21.342954125488355, 21.313159922183605, 21.286394219318467, 21.277013638626386, 21.253600667965316, 21.218195939559894, 21.19106675909008, 21.122590329130386, 21.007404857863147, 20.910400963341655, 20.85127913077661, 20.770555642938987, 20.687486709984288, 20.699832

In [12]:
mat = read_pdb(pdbIDs[4])
mat = np.array(mat).reshape((10,25,3)) # reshape the pdb to contain 10 models in a 3d matrix
for i in range(mat.shape[0]):
  A = mat[i]
  for j in range(i+1,mat.shape[0]):
    B = mat[j]
    A -= np.mean(A, axis = 0)
    B -= np.mean(B, axis = 0)

    U = rmsd.kabsch(A, B)
    A = np.dot(A, U)
    print(f"RMSD between {i+1} model and {j+1} model {calculate_rmsd(A, B)}")
    


RMSD between 1 model and 2 model 1.3509598986729614
RMSD between 1 model and 3 model 1.2695622138199805
RMSD between 1 model and 4 model 2.0038599950078044
RMSD between 1 model and 5 model 1.5726327036221934
RMSD between 1 model and 6 model 1.2566548110343225
RMSD between 1 model and 7 model 1.6432313305777357
RMSD between 1 model and 8 model 1.7644803780076173
RMSD between 1 model and 9 model 2.0669773859669256
RMSD between 1 model and 10 model 0.9248669123874687
RMSD between 2 model and 3 model 1.1656728704980588
RMSD between 2 model and 4 model 1.8436930218669043
RMSD between 2 model and 5 model 1.067448453139926
RMSD between 2 model and 6 model 2.0000532754503957
RMSD between 2 model and 7 model 1.5773946718512994
RMSD between 2 model and 8 model 1.7493154132555584
RMSD between 2 model and 9 model 2.9174105784137496
RMSD between 2 model and 10 model 1.6250498630069006
RMSD between 3 model and 4 model 1.974493794871596
RMSD between 3 model and 5 model 0.8457905038410172
RMSD between